# ZillizとOpenAIを使ったフィルタ検索
### あなたの次の映画を見つける

このノートブックでは、OpenAIを使って映画の説明文の埋め込みを生成し、それらの埋め込みをZilliz内で使用して関連する映画を見つける方法について説明します。検索結果を絞り込み、新しいことを試すために、メタデータ検索を行うフィルタリングを使用します。この例で使用するデータセットはHuggingFace datasetsから取得したもので、8千を少し超える映画エントリが含まれています。

まず、このノートブックに必要なライブラリをダウンロードすることから始めましょう：
- `openai`はOpenAI埋め込みサービスとの通信に使用されます
- `pymilvus`はZillizサーバーとの通信に使用されます
- `datasets`はデータセットのダウンロードに使用されます
- `tqdm`はプログレスバーに使用されます

In [ ]:
! pip install openai pymilvus datasets tqdm

Zillizを起動して実行するには、[こちら](https://zilliz.com/doc/quick_start)をご覧ください。アカウントとデータベースの設定が完了したら、以下の値を設定してください：
- URI: データベースが実行されているURI
- USER: データベースのユーザー名
- PASSWORD: データベースのパスワード
- COLLECTION_NAME: Zilliz内でコレクションに付ける名前
- DIMENSION: 埋め込みの次元数
- OPENAI_ENGINE: 使用する埋め込みモデル
- openai.api_key: OpenAIアカウントキー
- INDEX_PARAM: コレクションに使用するインデックス設定
- QUERY_PARAM: 使用する検索パラメータ
- BATCH_SIZE: 一度に埋め込みと挿入を行うテキストの数

In [1]:
import openai

URI = 'your_uri'
TOKEN = 'your_token' # TOKEN == user:password or api_key
COLLECTION_NAME = 'book_search'
DIMENSION = 1536
OPENAI_ENGINE = 'text-embedding-3-small'
openai.api_key = 'sk-your_key'

INDEX_PARAM = {
    'metric_type':'L2',
    'index_type':"AUTOINDEX",
    'params':{}
}

QUERY_PARAM = {
    "metric_type": "L2",
    "params": {},
}

BATCH_SIZE = 1000

In [3]:
from pymilvus import connections, utility, FieldSchema, Collection, CollectionSchema, DataType

# Connect to Zilliz Database
connections.connect(uri=URI, token=TOKEN)

In [4]:
# Remove collection if it already exists
if utility.has_collection(COLLECTION_NAME):
    utility.drop_collection(COLLECTION_NAME)

In [5]:
# Create collection which includes the id, title, and embedding.
fields = [
    FieldSchema(name='id', dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name='title', dtype=DataType.VARCHAR, max_length=64000),
    FieldSchema(name='type', dtype=DataType.VARCHAR, max_length=64000),
    FieldSchema(name='release_year', dtype=DataType.INT64),
    FieldSchema(name='rating', dtype=DataType.VARCHAR, max_length=64000),
    FieldSchema(name='description', dtype=DataType.VARCHAR, max_length=64000),
    FieldSchema(name='embedding', dtype=DataType.FLOAT_VECTOR, dim=DIMENSION)
]
schema = CollectionSchema(fields=fields)
collection = Collection(name=COLLECTION_NAME, schema=schema)

In [6]:
# Create the index on the collection and load it.
collection.create_index(field_name="embedding", index_params=INDEX_PARAM)
collection.load()

## データセット
Zillizが稼働している状態で、データの取得を開始できます。`Hugging Face Datasets`は多くの異なるユーザーデータセットを保持するハブであり、この例ではHuggingLearnersのnetflix-showsデータセットを使用しています。このデータセットには8千以上の映画とそのメタデータのペアが含まれています。各説明文を埋め込み、タイトル、タイプ、リリース年、評価と共にZilliz内に保存します。

In [7]:
import datasets

# Download the dataset 
dataset = datasets.load_dataset('hugginglearners/netflix-shows', split='train')

/Users/filiphaltmayer/miniconda3/envs/haystack/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Found cached dataset csv (/Users/filiphaltmayer/.cache/huggingface/datasets/hugginglearners___csv/hugginglearners--netflix-shows-03475319fc65a05a/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


## データの挿入
データがマシン上に準備できたので、埋め込み処理を行ってZillizに挿入を開始できます。埋め込み関数はテキストを受け取り、リスト形式で埋め込みを返します。

In [8]:
# Simple function that converts the texts to embeddings
def embed(texts):
    embeddings = openai.Embedding.create(
        input=texts,
        engine=OPENAI_ENGINE
    )
    return [x['embedding'] for x in embeddings['data']]


次のステップでは実際の挿入を行います。すべてのエントリを反復処理し、設定されたバッチサイズに達したらバッチを作成して挿入します。ループが終了した後、残りの最後のバッチが存在する場合はそれを挿入します。

In [9]:
from tqdm import tqdm

data = [
    [], # title
    [], # type
    [], # release_year
    [], # rating
    [], # description
]

# Embed and insert in batches
for i in tqdm(range(0, len(dataset))):
    data[0].append(dataset[i]['title'] or '')
    data[1].append(dataset[i]['type'] or '')
    data[2].append(dataset[i]['release_year'] or -1)
    data[3].append(dataset[i]['rating'] or '')
    data[4].append(dataset[i]['description'] or '')
    if len(data[0]) % BATCH_SIZE == 0:
        data.append(embed(data[4]))
        collection.insert(data)
        data = [[],[],[],[],[]]

# Embed and insert the remainder 
if len(data[0]) != 0:
    data.append(embed(data[4]))
    collection.insert(data)
    data = [[],[],[],[],[]]


100%|██████████| 8807/8807 [00:54<00:00, 162.59it/s]


## データベースのクエリ
データがZillizに安全に挿入されたので、クエリを実行できるようになりました。クエリは、検索したい映画の説明と使用するフィルターのタプルを受け取ります。フィルターの詳細については[こちら](https://milvus.io/docs/boolean.md)をご覧ください。検索では、まず説明とフィルター式が出力されます。その後、各結果について、結果の映画のスコア、タイトル、タイプ、公開年、評価、説明が出力されます。

In [10]:
import textwrap

def query(query, top_k = 5):
    text, expr = query
    res = collection.search(embed(text), anns_field='embedding', expr = expr, param=QUERY_PARAM, limit = top_k, output_fields=['title', 'type', 'release_year', 'rating', 'description'])
    for i, hit in enumerate(res):
        print('Description:', text, 'Expression:', expr)
        print('Results:')
        for ii, hits in enumerate(hit):
            print('\t' + 'Rank:', ii + 1, 'Score:', hits.score, 'Title:', hits.entity.get('title'))
            print('\t\t' + 'Type:', hits.entity.get('type'), 'Release Year:', hits.entity.get('release_year'), 'Rating:', hits.entity.get('rating'))
            print(textwrap.fill(hits.entity.get('description'), 88))
            print()

my_query = ('movie about a fluffly animal', 'release_year < 2019 and rating like \"PG%\"')

query(my_query)

Description: movie about a fluffly animal Expression: release_year < 2019 and rating like "PG%"
Results:
	Rank: 1 Score: 0.30085673928260803 Title: The Lamb
		Type: Movie Release Year: 2017 Rating: PG
A big-dreaming donkey escapes his menial existence and befriends some free-spirited
animal pals in this imaginative retelling of the Nativity Story.

	Rank: 2 Score: 0.3352621793746948 Title: Puss in Boots
		Type: Movie Release Year: 2011 Rating: PG
The fabled feline heads to the Land of Giants with friends Humpty Dumpty and Kitty
Softpaws on a quest to nab its greatest treasure: the Golden Goose.

	Rank: 3 Score: 0.3415083587169647 Title: Show Dogs
		Type: Movie Release Year: 2018 Rating: PG
A rough and tough police dog must go undercover with an FBI agent as a prim and proper
pet at a dog show to save a baby panda from an illegal sale.

	Rank: 4 Score: 0.3428957462310791 Title: Open Season 2
		Type: Movie Release Year: 2008 Rating: PG
Elliot the buck and his forest-dwelling cohorts must